In [1]:
import pandas as pd
import numpy as np
# pandas와 numpy 모델 호출. DataFrame과 산술 작업 예정.

In [2]:
from sklearn import tree
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
# ensemble.RandomForestRegressor()
# tree.DecisionTreeRegressor()

In [3]:
featuresData = pd.read_csv("../../../stdcode6/dataset/feature_regression_example.csv")

# A. 데이터 전처리

## A-1. 타입 통합 / 특성 숫자컬럼 추가

In [4]:
featuresData.head(5)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155


In [5]:
featuresData.dtypes
# featuresData 데이터프레임의 각 데이터 타입 조회

REGIONID         object
PRODUCTGROUP     object
PRODUCT          object
ITEM             object
YEARWEEK          int64
YEAR              int64
WEEK              int64
QTY               int64
HOLIDAY          object
HCLUS             int64
PROMOTION        object
PRO_PERCENT     float64
dtype: object

In [6]:
featuresData = featuresData.astype({"YEARWEEK":int,
                                  "YEAR":int,
                                  "WEEK":int})
# YEARWEEK, YEAR, WEEK 컬럼의 데이터 타입을 int로 형변환
# 현재도 int64라 굳이 필요한가 싶은 단계

In [7]:
featuresData.HOLIDAY.drop_duplicates()

0    Y
1    N
Name: HOLIDAY, dtype: object

In [8]:
featuresData.PROMOTION.drop_duplicates()

0    Y
9    N
Name: PROMOTION, dtype: object

In [9]:
featuresData["HO_YN"] = np.where(featuresData.HOLIDAY == "Y", 1, 0)
# HO_YN 컬럼을 신설하고, HOLIDAY가 Y라면 1을, 아니라면 0을 값으로 산입한다.
featuresData["PRO_YN"] = np.where(featuresData.PROMOTION == "Y", 1, 0)
# PRO_YN 컬럼을 신설하고, PROMOTION이 Y라면 1을, 아니라면 0을 값으로 산입한다.
featuresData.head(10)
# 상부 Row 10개를 가져와 제대로 들어갔는지 확인

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442,0,1
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155,0,1
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155,0,1
5,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201506,2015,6,867,N,4,Y,0.208155,0,1
6,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201507,2015,7,1187,N,4,Y,0.208155,0,1
7,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201508,2015,8,970,Y,1,Y,0.208155,1,1
8,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201509,2015,9,542,N,4,Y,0.208155,0,1
9,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201510,2015,10,350,N,4,N,0.000000,0,0


In [ ]:
# Y, N로 구성되어 있던 HOLIDAY와 

In [10]:
featuresData.loc[(featuresData.HOLIDAY == "Y") &
                 (featuresData.HO_YN != 1)]
# 반증 사례 확인을 통해 적용되지 않은 값이 존재하는지 파악

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN


In [11]:
featuresData.loc[(featuresData.PROMOTION == "Y") &
                 (featuresData.PRO_YN != 1)]
# 마찬가지로 반증 사례 검증을 통해 예외값의 존재 유무를 파악

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN


In [12]:
featuresData.head(1)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442,1,1


In [13]:
featuresData.drop_duplicates()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,HO_YN,PRO_YN
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442,0,1
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155,0,1
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888,1,1
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888,0,1
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361,0,1
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361,0,1


## A-2 특성 선정 / 데이터 분리

In [14]:
corrDf = featuresData.corr(method="pearson")

In [15]:
corrDf

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT,HO_YN,PRO_YN
YEARWEEK,1.000000,0.956598,0.275593,0.112267,-0.071586,0.404889,0.049867,0.205916
YEAR,0.956598,1.000000,-0.016493,0.028931,0.028593,0.321193,-0.031106,0.195931
WEEK,0.275593,-0.016493,1.000000,0.289766,-0.339943,0.329705,0.273710,0.060206
QTY,0.112267,0.028931,0.289766,1.000000,-0.537230,0.700195,0.505932,0.612451
HCLUS,-0.071586,0.028593,-0.339943,-0.537230,1.000000,-0.545619,-0.974601,-0.374072
PRO_PERCENT,0.404889,0.321193,0.329705,0.700195,-0.545619,1.000000,0.487062,0.898554
HO_YN,0.049867,-0.031106,0.273710,0.505932,-0.974601,0.487062,1.000000,0.365148
PRO_YN,0.205916,0.195931,0.060206,0.612451,-0.374072,0.898554,0.365148,1.000000


In [ ]:
# pearson : 두 변수의 ABS(상관관계)의 값. 상하방으로 비슷한 기울기를 지니면 1에 수렴
# 일반적으로 상관관계가 선형성을 지니는 데이터에 적합하다.
# 가장 사용 빈도가 높은 방식이며, 이외에도 spearman과 kendall 방식이 존재한다.

In [16]:
featureStd = 0.5
# 상관계수 인식 기준으로 0.5을 설정

In [17]:
features = list(corrDf.loc[(abs(corrDf.QTY) > featureStd) &
                           (abs(corrDf.QTY) != 1)].index)
# 상관관계 == 1 : 자기 자신을 의미함.
# 이에 1이 아니며, 0.5보다 높은 상관계수 절댓값을 지니는 컬럼을 파악한다.

In [ ]:
# abs(절댓값)을 사용하여 상관계수의 크기만을 고려 대상으로 설정한다. 음양은 상관없다.

In [18]:
features
# 영향을 주는 요인들을 features에 산입

['HCLUS', 'PRO_PERCENT', 'HO_YN', 'PRO_YN']

In [19]:
label = ['QTY']

In [20]:
label

['QTY']

In [21]:
yearweekStd = 201630
# test데이터와 train데이터의 기준을 201630주차 데이터를 기준으로 나눴다.

In [22]:
trainingData_feature = \
featuresData.loc[featuresData.YEARWEEK <= yearweekStd][features]
trainingData_label = \
featuresData.loc[featuresData.YEARWEEK <= yearweekStd][label]

In [23]:
trainingData_feature
# 학습 대상으로 삼을 feature 데이터들

,HCLUS,PRO_PERCENT,HO_YN,PRO_YN
0,1,0.209442,1,1
1,4,0.209442,0,1
2,4,0.208155,0,1
3,2,0.208155,1,1
4,4,0.208155,0,1
...,...,...,...,...
78,1,0.308584,1,1
79,1,0.308584,1,1
80,1,0.308584,1,1
81,1,0.308584,1,1


In [24]:
trainingData_label
# 학습 대상으로 삼을 label 데이터들

,QTY
0,1225
1,968
2,1209
3,1810
4,1773
...,...
78,1685
79,1586
80,1616
81,1318


In [25]:
testData_feature = \
featuresData.loc[featuresData.YEARWEEK > yearweekStd][features]
testData_label = \
featuresData.loc[featuresData.YEARWEEK > yearweekStd][label]

In [26]:
testData_feature
# 검증 간 활용할 features데이터들

,HCLUS,PRO_PERCENT,HO_YN,PRO_YN
83,4,0.280258,0,1
84,4,0.280258,0,1
85,4,0.000000,0,0
86,1,0.308584,1,1
87,1,0.308584,1,1
88,1,0.308584,1,1
89,4,0.308584,0,1
90,4,0.308584,0,1
91,4,0.280258,0,1
92,4,0.280258,0,1


In [27]:
testData_label
# 검증 대상으로 삼을 label 데이터들

,QTY
83,1522
84,2100
85,43
86,1700
87,1514
88,1501
89,1491
90,806
91,2111
92,2400


In [28]:
print(trainingData_feature.shape)
print(trainingData_label.shape)
print(testData_feature.shape)
print(testData_label.shape)

(83, 4)
(83, 1)
(22, 4)
(22, 1)


In [ ]:
# 동일한 행 갯수를 지니는지 파악. 위 데이터는 제대로 구분이 이뤄졌다.

## B-3. 모델 적용

In [29]:
model_method = \
tree.DecisionTreeRegressor(random_state=10)
# 모델 정의
# random_state를 고정하면 일관적인 답이 도출된다...?
# 동일한 데이터에는 동일한 값이 도출된다.

In [30]:
model = model_method.fit(X = trainingData_feature,
                         y = trainingData_label)
# 추후 pickle 파일로 저장 가능하다.

# C.예측

## C-1. 예측

In [31]:
predict = model.predict(X = testData_feature)
predict

array([1606.5       , 1606.5       ,  350.71428571, 1434.6       ,
       1434.6       , 1434.6       , 1434.6       , 1434.6       ,
       1606.5       , 1606.5       , 2620.42857143, 1606.5       ,
        350.71428571, 2193.8       , 2193.8       , 2193.8       ,
       2193.8       , 2193.8       , 1434.6       , 1434.6       ,
       1434.6       , 1434.6       ])

In [32]:
predictData = pd.DataFrame(predict, columns = ["PREDICT"])

In [33]:
predictData

,PREDICT
0,1606.500000
1,1606.500000
2,350.714286
3,1434.600000
4,1434.600000
5,1434.600000
6,1434.600000
7,1434.600000
8,1606.500000
9,1606.500000


In [34]:
testData_label.reset_index(drop = True, inplace = True)

In [35]:
finalResult = pd.concat([testData_label, predictData], axis=1)

In [36]:
finalResult

,QTY,PREDICT
0,1522,1606.500000
1,2100,1606.500000
2,43,350.714286
3,1700,1434.600000
4,1514,1434.600000
5,1501,1434.600000
6,1491,1434.600000
7,806,1434.600000
8,2111,1606.500000
9,2400,1606.500000


In [37]:
pMae = mean_absolute_error(y_true = finalResult["QTY"],
                           y_pred = finalResult["PREDICT"])

In [38]:
pMae

402.7435064935065

In [39]:
pRmse = np.sqrt(mean_absolute_error(y_true = finalResult["QTY"],
                                    y_pred = finalResult["PREDICT"]))

In [40]:
pRmse

20.068470457249763

In [42]:
features

['HCLUS', 'PRO_PERCENT', 'HO_YN', 'PRO_YN']

In [52]:
inputHclus = 4 # 대휴일
inputProPercent = 0.5 # 프로모션 비율
inputPromotionLb = 0 # 프로모션 적용 Y
inputHlb = 1 # 홀리데이 적용 Y

In [53]:
testData = pd.DataFrame([[inputHclus, inputProPercent, inputPromotionLb, inputHlb]])

In [54]:
model.predict(testData)

C:\Users\SMART16\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([1434.6])